In [1]:
import pandas as pd

In [9]:
# Defining a dictionary that contains all the datasets
files = {
    "Joint_Malnutrition_Estimates": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\Joint-Malnutrition-Estimates-Regional-and-Global-Estimates-May-2023.xlsx",
    "FAOSTAT_Agriculture": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\FAOSTAT_Agriculture.csv",
    "FAOSTAT_Food_Insecurity": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\FAOSTAT_Food_Insecurity.csv",
    "World_Bank_CPI": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Consumer_Price_Index.csv",
    "World_Bank_Fertility_Rate" :"C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Fertility_Rate.csv",
    "World_Bank_GDP_Per_Capita": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_GDP_Per_Capita.csv",
    "World_Bank_Population_Total": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Population_Total.csv",
    "World_Bank_Population_Growth": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Population_Growth.csv",
    "World_Bank_Poverty_at_3.65": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Poverty_at_3.65$.csv",
    "World_Bank_Poverty_at_Societal_Line": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Poverty_at_Societal_Line.csv"
}